In [6]:
from openai import OpenAI 
import yaml
import os
from dotenv import load_dotenv

load_dotenv()

MODEL="gpt-4o"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>")
DB_INDEX_FILE_LOCAL_PATH = os.environ.get("DB_INDEX_FILE_LOCAL_PATH", "<path to the index.json file>")
DB_ROOT_LOCAL_PATH = os.environ.get("DB_ROOT_LOCAL_PATH", "<path to the root directory of the database>")

openAiClient = OpenAI(api_key=OPENAI_API_KEY)

with open("../prompts/memoryClassifier.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_classifier = group_classifier["prompt"]

with open("../prompts/memoryAssistant.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_assistant = group_classifier["prompt"]

with open(DB_INDEX_FILE_LOCAL_PATH, "r") as file:
    metadata = file.read()

In [7]:
def create_messages_for_classifier(query, metadata):
    return [
        {
            "role": "system",
            "content": system_prompt_for_classifier
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Metadata: ${metadata}"
        }        
    ]

def create_messages_for_assistant(query, memories):
    return [
        {
            "role": "system",
            "content": system_prompt_for_assistant
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Memories: ${memories}"
        }
    ]

In [12]:
query = "any movie to recommend?"
completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_classifier(query, metadata)),

completion_content = completion[0].choices[0].message.content
print(completion_content)

{
  "thinking_process": "The user's query is about movie recommendations, which falls under the category of entertainment. Based on the metadata provided, the most relevant file is 'AREAS/ENTERTAINMENT/tv_shows', which contains a list of TV shows the user has watched. Although it primarily lists TV shows, it might also include references to movies or similar entertainment content.",
  "keys": [
    "AREAS/ENTERTAINMENT/tv_shows"
  ]
}


In [13]:
# parse the response in JSON and print the value of *data*
import json


keys = json.loads(completion_content)["keys"]

memories = []
for key in keys:
    memory_asset_path = os.path.join(DB_ROOT_LOCAL_PATH, key)
    with open(memory_asset_path, "r") as file:
        memory_asset_content = file.read()
        memories.append({
            "path": key,
            "content": memory_asset_content
        })

memories_string = json.dumps(memories)

completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_assistant(query, memories_string)),

completion_content = completion[0].choices[0].message.content
print(completion_content)    


{
  "thinking_process": "The user has asked for a movie recommendation. The provided memories list TV shows the user has watched, which gives insight into their preferences. Based on the genres and themes of these shows, I can suggest movies that align with their interests.",
  "response": "Based on the TV shows you've enjoyed, here are a few movie recommendations:\n\n1. **Inception** - If you liked 'Westworld' and 'Black Mirror', you'll enjoy this mind-bending sci-fi thriller.\n2. **The Social Network** - Given your interest in 'Billions' and 'Succession', this film about the founding of Facebook might intrigue you.\n3. **The Grand Budapest Hotel** - If you enjoyed 'The Crown' and 'Downton Abbey', this quirky, visually stunning film by Wes Anderson could be a great pick.\n4. **Parasite** - If you liked 'Ozark' and 'Breaking Bad', this dark, thrilling, and critically acclaimed film might be right up your alley.\n5. **Knives Out** - If you enjoyed 'Broadchurch' and 'Mare of Easttown', t